# Analysis of Orphan Wells in Texas

## Overview

This notebook analyzes the distribution and regulatory performance of orphan wells in Texas. The analysis focuses on two main aspects:
1. The distribution of plugged and unplugged orphan wells.
2. The inspection and violation history of these wells.
3. The spatial distribution of orphan wells across different districts and district offices.
4. The most recent inspection and violation dates for orphan wells.
5. The operators and districts with the highest number of orphan wells, both plugged and unplugged.

## Data Sources

- **Well Data**: Contains information about wells including their API number, operator, lease name, completion date, and plug date.
- **Well Shapes**: Geospatial data of wells.
- **Orphaned Wells Shape**: Contains information about orphaned wells including a unique identifier (`well_ident`).
- **Inspections**: Contains details about inspections including `inspection_date`, `operator_name`, `district`, and `district_office_inspecting`.
- **Violations**: Contains details about violations including `violation_disc_date`.
- **Well Inspections**: Maps wells to their inspections.
- **Well Violations**: Maps wells to their violations.

## Key Steps and Analysis

### 1. Data Preparation

- **Standardized API Numbers**: Extracted 8-digit API numbers from the `well_ident` field in the orphaned wells dataset.
- **Merged Data**: Combined well data with shapes and orphaned wells data to create a detailed dataset of orphan wells.
- **Filtered Data**: Separated the dataset into plugged and unplugged wells for focused analysis.

### 2. Inspection and Violation Data

- **Merged Inspection and Violation Data**: Incorporated inspection and violation dates into the orphan wells dataset.
- **Handled Missing/Invalid Dates**: Cleaned the data to ensure valid dates for meaningful analysis.

### 3. Visualization

- **Distribution of Plugged and Unplugged Wells**: Created maps to visualize the spatial distribution of plugged and unplugged orphan wells in Texas.
- **Inspection and Violation Analysis**: Analyzed the most recent inspection and violation dates, and visualized the data by district and district office.

## Summary of Findings

### Spatial Distribution

- **Plugged Wells**: A small number of orphan wells have been plugged.
- **Unplugged Wells**: A significant number of orphan wells remain unplugged.

### Inspection and Violation History

- **Inspection Dates**: The distribution of the most recent inspection dates varies across different districts and district offices, with some areas showing more recent activity.
- **Violation Dates**: The distribution of violation dates also varies, highlighting areas where regulatory enforcement may be more active.



---

In [3]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text
import os
from shapely.geometry import Point


# Get environment variables for PostgreSQL connection
db_user = os.environ.get('DB_USER')
db_password = os.environ.get('DB_PASSWORD') 
db_host = os.environ.get('DB_HOST')
db_port = os.environ.get('DB_PORT')
 

# Create a connection to the PostgreSQL database
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/texas_oil')

# Load the datasets
well_data = pd.read_sql("SELECT * FROM well_data", engine)
well_shapes = gpd.read_postgis("SELECT * FROM well_shapes", engine, geom_col='geometry')
orphaned_wells_shape = gpd.read_postgis("SELECT * FROM orphaned_wells_shape", engine, geom_col='geom')

In [ ]:
# Extract the 8-digit API number from well_ident in orphaned_wells_shape
orphaned_wells_shape['api_no'] = orphaned_wells_shape['well_ident'].str[9:17]

# Confirm the changes
print(orphaned_wells_shape['api_no'].str.len().value_counts())


In [ ]:
# Merge well_data with well_shapes
well_data_merged = well_data.merge(well_shapes, left_on='api_num', right_on='api', how='left')

# Merge orphaned wells with well data
orphan_wells_detailed = orphaned_wells_shape.merge(well_data_merged, left_on='api_no', right_on='api_num', how='left')

# Check the number of rows in the merged dataset
print(f"Number of rows in orphan_wells_detailed: {orphan_wells_detailed.shape[0]}")


In [ ]:
# Filter for plugged and unplugged orphan wells and make explicit copies
plugged_wells = orphan_wells_detailed[~orphan_wells_detailed['plug_date'].isna()].copy()
unplugged_wells = orphan_wells_detailed[orphan_wells_detailed['plug_date'].isna()].copy()

# Confirm the shapes of the DataFrames
print(f"Plugged wells: {plugged_wells.shape}")
print(f"Unplugged wells: {unplugged_wells.shape}")


In [ ]:
from shapely.geometry import Point

# Ensure the geometry is of type Point
plugged_wells['geometry'] = plugged_wells['geom'].apply(lambda geom: Point(geom) if not isinstance(geom, Point) else geom)
unplugged_wells['geometry'] = unplugged_wells['geom'].apply(lambda geom: Point(geom) if not isinstance(geom, Point) else geom)

# Confirm the conversion
print(plugged_wells['geometry'].head())
print(unplugged_wells['geometry'].head())


In [ ]:
# Ensure api_no columns are strings of 8 digits
plugged_wells['api_no'] = plugged_wells['api_no'].astype(str).str.zfill(8)
unplugged_wells['api_no'] = unplugged_wells['api_no'].astype(str).str.zfill(8)


In [ ]:
from sqlalchemy import text

# Create SQL commands to create the tables
create_plugged_wells_table = """
CREATE TABLE IF NOT EXISTS plugged_orphan_wells (
    id SERIAL PRIMARY KEY,
    api_no VARCHAR(8),
    plug_date DATE,
    geometry GEOMETRY(Point, 4326),
    operator VARCHAR,
    lease_name VARCHAR,
    completion_date DATE
);
"""

create_unplugged_wells_table = """
CREATE TABLE IF NOT EXISTS unplugged_orphan_wells (
    id SERIAL PRIMARY KEY,
    api_no VARCHAR(8),
    plug_date DATE,
    geometry GEOMETRY(Point, 4326),
    operator VARCHAR,
    lease_name VARCHAR,
    completion_date DATE
);
"""

# Execute the SQL commands
with engine.connect() as conn:
    conn.execute(text(create_plugged_wells_table))
    conn.execute(text(create_unplugged_wells_table))

# Write the DataFrames to the PostgreSQL database
plugged_wells.to_postgis('plugged_orphan_wells', engine, if_exists='replace', index=False)
unplugged_wells.to_postgis('unplugged_orphan_wells', engine, if_exists='replace', index=False)

print("Data inserted into PostgreSQL tables successfully.")


In [ ]:
# Check if the data has been correctly inserted into the plugged_orphan_wells table
plugged_wells_in_db = gpd.read_postgis("SELECT * FROM plugged_orphan_wells", engine, geom_col='geom')
print(f"Plugged wells in database: {plugged_wells_in_db.shape}")

# Check if the data has been correctly inserted into the unplugged_orphan_wells table
unplugged_wells_in_db = gpd.read_postgis("SELECT * FROM unplugged_orphan_wells", engine, geom_col='geom')
print(f"Unplugged wells in database: {unplugged_wells_in_db.shape}")


In [ ]:
import matplotlib.pyplot as plt

# Plot the plugged and unplugged wells
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
plugged_wells_in_db.plot(ax=ax, color='blue', marker='o', label='Plugged Wells')
unplugged_wells_in_db.plot(ax=ax, color='red', marker='x', label='Unplugged Wells')
plt.title('Plugged and Unplugged Orphan Wells in Texas', fontsize=20)
plt.xlabel('Longitude', fontsize=15)
plt.ylabel('Latitude', fontsize=15)
plt.legend()
plt.show()


In [ ]:
import seaborn as sns

# Plotting the spatial distribution using seaborn for better clarity
plt.figure(figsize=(15, 10))
sns.scatterplot(x=plugged_wells_in_db.geometry.x, y=plugged_wells_in_db.geometry.y, color='blue', marker='o', label='Plugged Wells')
sns.scatterplot(x=unplugged_wells_in_db.geometry.x, y=unplugged_wells_in_db.geometry.y, color='red', marker='x', label='Unplugged Wells')
plt.title('Plugged and Unplugged Orphan Wells in Texas', fontsize=20)
plt.xlabel('Longitude', fontsize=15)
plt.ylabel('Latitude', fontsize=15)
plt.legend()
plt.show()


In [ ]:
# Basic summary statistics
plugged_wells_summary = plugged_wells.describe()
unplugged_wells_summary = unplugged_wells.describe()

print("Summary Statistics for Plugged Wells:")
print(plugged_wells_summary)

print("\nSummary Statistics for Unplugged Wells:")
print(unplugged_wells_summary)


In [ ]:
# Distribution of plugged wells by operator
plugged_operator_distribution = plugged_wells['operator'].value_counts()
print("Plugged Wells by Operator:")
print(plugged_operator_distribution)

# Distribution of unplugged wells by operator
unplugged_operator_distribution = unplugged_wells['operator'].value_counts()
print("\nUnplugged Wells by Operator:")
print(unplugged_operator_distribution)


In [ ]:
# Load the datasets
well_inspections = pd.read_sql("SELECT * FROM well_inspections", engine)
well_violations = pd.read_sql("SELECT * FROM well_violations", engine)
inspections = pd.read_sql("SELECT * FROM inspections", engine)
violations = pd.read_sql("SELECT * FROM violations", engine)


In [ ]:
# Standardize API numbers in well_inspections and well_violations
well_inspections['api_well_id'] = well_inspections['api_well_id'].astype(str).str.zfill(8)
well_violations['api_well_id'] = well_violations['api_well_id'].astype(str).str.zfill(8)


In [ ]:
# Merge unplugged wells with well_inspections to get inspection IDs
unplugged_wells_with_inspections = unplugged_wells.merge(well_inspections, left_on='api_no', right_on='api_well_id', how='left')

# Merge unplugged wells with well_violations to get violation IDs
unplugged_wells_with_violations = unplugged_wells.merge(well_violations, left_on='api_no', right_on='api_well_id', how='left')

# Merge unplugged_wells_with_inspections with inspections to get inspection dates
unplugged_wells_with_inspection_dates = unplugged_wells_with_inspections.merge(
    inspections[['id', 'inspection_date']], 
    left_on='inspection_id', right_on='id', how='left', suffixes=('', '_inspection'))

# Merge unplugged_wells_with_violations with violations to get violation dates
unplugged_wells_with_violation_dates = unplugged_wells_with_violations.merge(
    violations[['id', 'violation_disc_date']], 
    left_on='violation_id', right_on='id', how='left', suffixes=('', '_violation'))

# Drop the extra 'id' columns after merge
unplugged_wells_with_inspection_dates.drop(['id'], axis=1, inplace=True)
unplugged_wells_with_violation_dates.drop(['id'], axis=1, inplace=True)


In [ ]:
# Combine the inspection and violation data
unplugged_wells_full = unplugged_wells_with_inspection_dates.merge(
    unplugged_wells_with_violation_dates, on='api_no', how='outer', suffixes=('', '_violation'))

# Confirm the final DataFrame
print(unplugged_wells_full.columns)


In [ ]:
# Group by api_no to find the most recent inspection and violation dates
most_recent_inspections = unplugged_wells_full.groupby('api_no')['inspection_date'].max().reset_index()
most_recent_violations = unplugged_wells_full.groupby('api_no')['violation_disc_date'].max().reset_index()

# Merge the most recent dates back to the unplugged_wells_full DataFrame
unplugged_wells_full = unplugged_wells_full.merge(most_recent_inspections, on='api_no', suffixes=('', '_most_recent_inspection'))
unplugged_wells_full = unplugged_wells_full.merge(most_recent_violations, on='api_no', suffixes=('', '_most_recent_violation'))

# Confirm the final DataFrame
print(unplugged_wells_full[['api_no', 'inspection_date_most_recent_inspection', 'violation_disc_date_most_recent_violation']].head())


In [ ]:
# Summary statistics for inspection dates of unplugged wells
inspection_date_summary_unplugged = unplugged_wells_full['inspection_date_most_recent_inspection'].describe()
print("Summary Statistics for Most Recent Inspection Dates (Unplugged Wells):")
print(inspection_date_summary_unplugged)

# Summary statistics for violation dates of unplugged wells
violation_date_summary_unplugged = unplugged_wells_full['violation_disc_date_most_recent_violation'].describe()
print("\nSummary Statistics for Most Recent Violation Dates (Unplugged Wells):")
print(violation_date_summary_unplugged)


In [ ]:
import matplotlib.pyplot as plt

# Plot the distribution of inspection dates for unplugged wells
plt.figure(figsize=(10, 6))
unplugged_wells_full['inspection_date_most_recent_inspection'].dropna().hist(bins=20)
plt.title('Distribution of Most Recent Inspection Dates (Unplugged Wells)')
plt.xlabel('Inspection Date')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Plot the distribution of violation dates for unplugged wells
plt.figure(figsize=(10, 6))
unplugged_wells_full['violation_disc_date_most_recent_violation'].dropna().hist(bins=20)
plt.title('Distribution of Most Recent Violation Dates (Unplugged Wells)')
plt.xlabel('Violation Date')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Merge unplugged wells with inspections to get operator names
unplugged_wells_with_operator = unplugged_wells_full.merge(
    inspections[['id', 'operator_name']], 
    left_on='inspection_id', right_on='id', how='left', suffixes=('', '_inspection'))

# Confirm the final DataFrame
print(unplugged_wells_with_operator[['api_no', 'inspection_date_most_recent_inspection', 'violation_disc_date_most_recent_violation', 'operator_name']].head())


In [ ]:
# Group by operator_name to get summary statistics for inspection dates
inspection_date_by_operator = unplugged_wells_with_operator.groupby('operator_name')['inspection_date_most_recent_inspection'].describe()
print("Summary Statistics for Most Recent Inspection Dates by Operator:")
print(inspection_date_by_operator)

# Group by operator_name to get summary statistics for violation dates
violation_date_by_operator = unplugged_wells_with_operator.groupby('operator_name')['violation_disc_date_most_recent_violation'].describe()
print("\nSummary Statistics for Most Recent Violation Dates by Operator:")
print(violation_date_by_operator)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Defragment the DataFrame
unplugged_wells_with_operator = unplugged_wells_with_operator.copy()

# Convert inspection dates to datetime if necessary
if unplugged_wells_with_operator['inspection_date_most_recent_inspection'].dtype != 'datetime64[ns]':
    unplugged_wells_with_operator['inspection_date_most_recent_inspection'] = pd.to_datetime(unplugged_wells_with_operator['inspection_date_most_recent_inspection'])

# Plot the distribution of inspection dates by operator for unplugged wells
plt.figure(figsize=(15, 10))
sns.histplot(data=unplugged_wells_with_operator, 
             x='inspection_date_most_recent_inspection', 
             hue='operator_name', 
             multiple='stack', 
             bins=20, 
             palette='Set1')

# Title and labels
plt.title('Distribution of Most Recent Inspection Dates by Operator (Unplugged Wells)', fontsize=16)
plt.xlabel('Inspection Date', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Customize legend
plt.legend(title='Operator', title_fontsize='13', fontsize='11', loc='upper right')

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Defragment the DataFrame
unplugged_wells_with_operator = unplugged_wells_with_operator.copy()

# Convert violation dates to datetime if necessary
if unplugged_wells_with_operator['violation_disc_date_most_recent_violation'].dtype != 'datetime64[ns]':
    unplugged_wells_with_operator['violation_disc_date_most_recent_violation'] = pd.to_datetime(unplugged_wells_with_operator['violation_disc_date_most_recent_violation'])

# Plot the distribution of violation dates by operator for unplugged wells
plt.figure(figsize=(15, 10))
sns.histplot(data=unplugged_wells_with_operator, 
             x='violation_disc_date_most_recent_violation', 
             hue='operator_name', 
             multiple='stack', 
             bins=20, 
             palette='Set1')

# Title and labels
plt.title('Distribution of Most Recent Violation Dates by Operator (Unplugged Wells)', fontsize=16)
plt.xlabel('Violation Date', fontsize=14)
plt.ylabel('Frequency', fontsize=14)

# Customize legend
plt.legend(title='Operator', title_fontsize='13', fontsize='11', loc='upper right')

# Display the plot
plt.tight_layout()
plt.show()


In [ ]:
# Merge unplugged wells with inspections to get district and district office information
unplugged_wells_with_district = unplugged_wells_full.merge(
    inspections[['id', 'operator_name', 'district', 'district_office_inspecting']], 
    left_on='inspection_id', right_on='id', how='left', suffixes=('', '_inspection'))

# Confirm the final DataFrame
print(unplugged_wells_with_district[['api_no', 'inspection_date_most_recent_inspection', 'violation_disc_date_most_recent_violation', 'operator_name', 'district', 'district_office_inspecting']].head())


In [ ]:
# Group by district to get summary statistics for inspection dates
inspection_date_by_district = unplugged_wells_with_district.groupby('district')['inspection_date_most_recent_inspection'].describe()
print("Summary Statistics for Most Recent Inspection Dates by District:")
print(inspection_date_by_district)

# Group by district office to get summary statistics for inspection dates
inspection_date_by_district_office = unplugged_wells_with_district.groupby('district_office_inspecting')['inspection_date_most_recent_inspection'].describe()
print("\nSummary Statistics for Most Recent Inspection Dates by District Office:")
print(inspection_date_by_district_office)

# Group by district to get summary statistics for violation dates
violation_date_by_district = unplugged_wells_with_district.groupby('district')['violation_disc_date_most_recent_violation'].describe()
print("\nSummary Statistics for Most Recent Violation Dates by District:")
print(violation_date_by_district)

# Group by district office to get summary statistics for violation dates
violation_date_by_district_office = unplugged_wells_with_district.groupby('district_office_inspecting')['violation_disc_date_most_recent_violation'].describe()
print("\nSummary Statistics for Most Recent Violation Dates by District Office:")
print(violation_date_by_district_office)


In [ ]:
# Convert dates to datetime
unplugged_wells_with_district['inspection_date_most_recent_inspection'] = pd.to_datetime(unplugged_wells_with_district['inspection_date_most_recent_inspection'])
unplugged_wells_with_district['violation_disc_date_most_recent_violation'] = pd.to_datetime(unplugged_wells_with_district['violation_disc_date_most_recent_violation'])

# Extract year from the dates
unplugged_wells_with_district['inspection_year'] = unplugged_wells_with_district['inspection_date_most_recent_inspection'].dt.year
unplugged_wells_with_district['violation_year'] = unplugged_wells_with_district['violation_disc_date_most_recent_violation'].dt.year


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Facet plot for inspection dates by district
g = sns.FacetGrid(unplugged_wells_with_district, col="district", col_wrap=4, height=4, sharey=False)
g.map_dataframe(sns.histplot, x="inspection_year", bins=15)
g.set_titles(col_template="District {col_name}")
g.set_axis_labels("Year", "Frequency")
g.fig.suptitle('Distribution of Most Recent Inspection Dates by District (Unplugged Wells)', y=1.02)
plt.show()


In [ ]:
# Facet plot for inspection dates by district office
g = sns.FacetGrid(unplugged_wells_with_district, col="district_office_inspecting", col_wrap=4, height=4, sharey=False)
g.map_dataframe(sns.histplot, x="inspection_year", bins=15)
g.set_titles(col_template="District Office {col_name}")
g.set_axis_labels("Year", "Frequency")
g.fig.suptitle('Distribution of Most Recent Inspection Dates by District Office (Unplugged Wells)', y=1.02)
plt.show()


In [ ]:
# Check for missing or invalid violation dates
print(unplugged_wells_with_district['violation_year'].isna().sum())
print(unplugged_wells_with_district['violation_year'].describe())

# Filter out rows with invalid violation dates (e.g., years far in the past or future)
valid_violation_years = unplugged_wells_with_district[unplugged_wells_with_district['violation_year'] > 1900]

# Confirm the changes
print(valid_violation_years['violation_year'].describe())


In [ ]:
# Facet plot for violation dates by district after cleaning
g = sns.FacetGrid(valid_violation_years, col="district", col_wrap=4, height=4, sharey=False)
g.map_dataframe(sns.histplot, x="violation_year", bins=15)
g.set_titles(col_template="District {col_name}")
g.set_axis_labels("Year", "Frequency")
g.fig.suptitle('Distribution of Most Recent Violation Dates by District (Unplugged Wells)', y=1.02)
plt.show()


In [ ]:
# Facet plot for violation dates by district office after cleaning
g = sns.FacetGrid(valid_violation_years, col="district_office_inspecting", col_wrap=4, height=4, sharey=False)
g.map_dataframe(sns.histplot, x="violation_year", bins=15)
g.set_titles(col_template="District Office {col_name}")
g.set_axis_labels("Year", "Frequency")
g.fig.suptitle('Distribution of Most Recent Violation Dates by District Office (Unplugged Wells)', y=1.02)
plt.show()
